In [ ]:
!pip uninstall -y tensorflow  # Remove TensorFlow
!pip install torch  # Install PyTorch
!pip install python-doctr[torch]  # Install docTR with PyTorch backend

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 820.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.5/299.5 kB 16.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=0a7ee8e74465823b342a44d329a753f326a99bd64d5208a4e4d01ea9c1e796bf
  Stored in directory: /root/.cache/

In [ ]:
%pip install python-doctr[torch]
%pip install fuzzywuzzy
%pip install requests pandas

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('train.csv')


# Display the first few rows of the dataframe
df.head()

# Assuming the CSV has a column 'file_path' with image paths
file_paths = df['image_link'].tolist()

In [ ]:
import requests
import pandas as pd
import os
from doctr.io import DocumentFile
import numpy
from doctr.models import ocr_predictor

# Initialize OCR model with vertical text handling
model = ocr_predictor(pretrained=True, assume_straight_pages=False, export_as_straight_boxes=True)

# Create a directory for downloaded images
image_dir = 'downloaded_image'
os.makedirs(image_dir, exist_ok=True)

# Load the CSV with URLs
df = pd.read_csv('train.csv')

# List to hold results
results = []

# Function to extract text from the OCR result
def extract_text(ocr_result):
    extracted_text = []
    # Traverse the result and extract the 'value' fields (words)
    for page in ocr_result['pages']:
        for block in page['blocks']:
            for line in block['lines']:
                for word in line['words']:
                    extracted_text.append(word['value'])
    # Join all words with a space
    return ' '.join(extracted_text)

# Iterate over each URL in the CSV
for idx, row in df.iterrows():
    image_url = row['image_link']  # Adjust column name if necessary

    try:
        # Download the image
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            # Create the local file path
            image_filename = os.path.join(image_dir, f"image_{idx}.jpg")
            with open(image_filename, 'wb') as img_file:
                img_file.write(response.content)

            # Perform OCR on the downloaded image
            doc = DocumentFile.from_images(image_filename)
            result = model(doc)
            extracted_text = extract_text(result.export())  # Extract only the text values
            results.append({'file_path': image_url, 'extracted_text': extracted_text})
        else:
            print(f"Failed to download {image_url}, status code: {response.status_code}")
    except Exception as e:
        print(f"Error processing {image_url}: {e}")

# Save the results to a CSV
output_df = pd.DataFrame(results)
output_df.to_csv('generated_text.csv', index=False)

print("OCR process completed. Results saved to 'generated_text.csv'.")

OCR process completed. Results saved to 'generated_text.csv'.


In [ ]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}


In [ ]:
!pip install textblob
!pip install scikit-learn
!pip install fuzzywuzzy

In [ ]:
!pip install transformers
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
import transformers
import accelerate

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

import torch
print(torch.__version__)

In [ ]:
import pandas as pd
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# Load the data
train_df = pd.read_csv('train.csv')
generate_df = pd.read_csv('generated_text.csv')

# Merge train_df with generate_df on the image_link column to access extracted_text
merged_df = pd.merge(train_df, generate_df, left_on='image_link', right_on='file_path', how='inner')

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Filter the text for numerical values and context
def filter_text(text):
    # Extract only sentences containing numerical values
    sentences = re.findall(r'([^.]*?\d+[^.]*\.)', text)
    return ' '.join(sentences)

# Preprocess data for training
def preprocess_data(df):
    texts = []
    labels = []

    for _, row in df.iterrows():
        extracted_text = filter_text(row['extracted_text'])
        entity_value = row['entity_value']
        
        # Label the text as 1 if it contains the entity value, otherwise 0
        if str(entity_value) in extracted_text:
            labels.append(1)
        else:
            labels.append(0)
        
        texts.append(extracted_text)

    return texts, labels

# Prepare the dataset
texts, labels = preprocess_data(merged_df)

# Tokenize the text data
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor(labels)

# Create a PyTorch dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = CustomDataset(inputs, labels)

# Split into training and evaluation sets
train_size = int(0.8 * len(dataset))
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Set up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Prediction function using BERT
def predict_entity_value(entity_name, extracted_text):
    filtered_text = filter_text(extracted_text)
    inputs = tokenizer(filtered_text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    
    # Return prediction if it's positive
    if prediction == 1:
        # Extract numerical value from filtered text
        candidates = re.findall(r'\d+(\.\d+)?\s*(gram|kilogram|milligram|ounce|pound|litre|cup|gallon|watt)', filtered_text)
        if candidates:
            return f"{candidates[0][0]} {candidates[0][1]}"
    return ""

# Perform predictions on `generate_text.csv`
predictions = []
for idx, row in generate_df.iterrows():
    file_path = row['file_path']
    extracted_text = row['extracted_text']
    
    # Find the entity name for this entry using the merged DataFrame
    train_row = merged_df[merged_df['file_path'] == file_path].iloc[0]
    entity_name = train_row['entity_name']
    actual_value = train_row['entity_value']
    
    # Predict the value
    predicted_value = predict_entity_value(entity_name, extracted_text)
    
    # Print debugging information
    print(f"Entity: {entity_name}, Entity Value: {actual_value}, Prediction: {predicted_value}, Extracted Text: {extracted_text}")
    
    predictions.append(predicted_value)

# Evaluate the predictions
output_df = pd.DataFrame({
    'index': generate_df.index,
    'prediction': predictions
})
output_df.to_csv('output_predictions.csv', index=False)

print("Prediction process completed.")